# Generate pseudo-WSIS from svs images

import packages

In [1]:
import numpy as np
from scipy.stats import norm

import matplotlib.pyplot as plt
from IPython.display import clear_output

import time
import sys
import os
import gc

In [3]:
sys.path.append('/home/jmalagont/Documentos/MLDS6project/src/nombre_paquete/preprocessing')
import pyWSI as pywsi

important variables

In [8]:
# path wich is stored the WSI
load_path = '/run/media/jmalagont/Thesis/Thesis/DataSet/TCGA-BRCA/WSI/'

# path where the pseudo-WSI will be stored
save_path = '/run/media/jmalagont/Thesis/Thesis/DataSet/TCGA-BRCA/pseudo-WSI/'

# number of patvches from each WSI
n_patches = 16

# patch size
patch_size = 64

# seed
seed = 1997
np.random.seed(seed)

#Verbose (0: No, 1: Only procces tracking, 2: Procces and image tracking)
verbose = 2

#Sleeping time between iterations 
sleeping_time = 3

## Pesudo WSI generation

In [9]:
WSI_names = os.listdir(load_path)
saved_pWSIs = os.listdir(save_path)

initial_time = time.time()
correct_executions = 0
wrong_executions = 0

for i in range (len(WSI_names)):
    WSI_name = WSI_names[i]
    print(f'Working on {WSI_name}')
    
    #1. WSI alrready processing?
    print(f' 1. WSI state: ', end = ' ')
    if (WSI_name[:-3] + 'npy') in saved_pWSIs:
        WSI_name = 'None'
        print('processed!')
    else:
        print('without treatment!')
        
    #2. Open WSI
    print(f' 2. Open WSI: ', end = ' ')
    try:
        slide = pywsi.svs_read(load_path + WSI_name)
        print('succesfully!')
    except:
        slide = None
        print('Error!')
    
    #3. Check maximum optical ampf
    print(f' 3. Maximum optical amplification: ', end = ' ')
    try:
        slide_properties = dict(slide.properties)
        maximum_mag = slide_properties['aperio.AppMag']
        print(f'X{maximum_mag}')
    except:
        maximum_mag = None
        print('None')
    
    #4. Compute reduction factor
    print(f' 4. Reduction factor: ', end = ' ')
    if maximum_mag == '40':
        reduction_factor = 2
    elif maximum_mag == '20':
        reduction_factor = 1
    else:
        reduction_factor = None
    print(f'{reduction_factor}')

    #5.  Relative patch size
    print(f' 5. Relative patch size for a {patch_size} patch: ', end = ' ')
    try:
        shapes = list(slide.level_dimensions)
        relative_patch_size = int((patch_size * reduction_factor) * (shapes[-1][0]/shapes[0][0])) + 2
        print(f'{round(relative_patch_size, 2)}')
    except:
        print('Error!')
    
    #6.  Extract ROI
    print(f' 6. Extract random ROI ({n_patches} patches, seed = {seed}): ', end = ' ')
    try:
        minimum_image = np.array(slide.read_region((0, 0), (len(shapes)-1), shapes[-1]).convert('RGB')) 
        ROI = pywsi.random_patch_roi(minimum_image, n_patches, [relative_patch_size, relative_patch_size], tissue_rate=.95, artifact_remotion=True, artifact_percentage_umbral=.1, seed=2011)
        print('succesfully!')
    except:
        print('Error!')
        
    #7.  Extract patches on X20 ampf
    print(f' 7. Extract high deffinition {patch_size} X {patch_size} patch: ', end = ' ')
    try:
        patches = pywsi.patch_extractions(slide, (len(shapes)-1), 0, ROI, magnitude_factor = reduction_factor, workers = None, verbose = False)
        patches = patches [:n_patches,:patch_size, :patch_size, :]
        print(f'succesfully!, extracted {patches.shape[0]} with shape {patches.shape[1]} X {patches.shape[2]}')
    except:
        patches = []
        print('Error!')

    #8.  Save the patches
    print(f' 8. Saving extracted patches: ', end = ' ')
    if np.size(patches) != 0:
        file_path =  save_path + WSI_name[:-3] + 'npy'
        np.save(file_path, patches, allow_pickle=False)
        slide.close()
        
        correct_executions = correct_executions + 1
        print(f'succesfully!')
    else:
        wrong_executions = wrong_executions + 1
        print('Error!')
    
    current_time = time.time()  
    executed_time = round(current_time - initial_time, 2)
    timeXexecution = round(executed_time/correct_executions, 2)
    remaining_executions = len(WSI_names) - i
    remaining_time = round(timeXexecution * remaining_executions, 2)
    
    #9. Verbose
    if verbose == 1:
        print()
        print('Execution summary:')
        print(f'Excuted {i+1} of {len(WSI_names)}')
        print(f'Excuted time: {executed_time}, iteration time:{timeXexecution},  remaining time: {remaining_time}')
        print(f'Correct executions:{correct_executions}, wrong executions: {wrong_executions}')
    elif verbose == 2:
        assambly = pywsi.patch_assembly(patches, assambly_size=None)
        print()
        print('Execution summary:')
        print(f'Excuted {i+1} of {len(WSI_names)}')
        print(f'Excuted time: {executed_time}, iteration time:{timeXexecution},  remaining time: {remaining_time}')
        print(f'Correct executions:{correct_executions}, wrong executions: {wrong_executions}')
        print()
        print('Images:')
        
        fig, char = plt.subplots(1,3)
        fig.set_size_inches(30, 7)

        char[0].imshow(minimum_image)
        char[0].axis('off')
        char[0].set_title('Image in X4 magnification')

        char[1].imshow(ROI)
        char[1].axis('off')
        char[1].set_title('ROI on the WSI')

        char[2].imshow(assambly)
        char[2].axis('off')
        char[2].set_title('Assambly')
        
        plt.show()
    else:
        pass
    
    time.sleep(sleeping_time)
    clear_output()

print('Execution summary:')
print(f'Excution time: {executed_time}, iteration time:{timeXexecution}')
print(f'Correct executions:{correct_executions}, wrong executions: {wrong_executions}')

Execution summary:
Excution time: 5020.55, iteration time:4.73
Correct executions:1062, wrong executions: 0
